In [1]:
#This notebook cleans and formats data from data.acgov.org and exports the csv

In [2]:
#This file includes the getAPN function which takes a string address as input and searches the acgov data for a matching parcel number.

In [3]:
import googlemaps
import pandas
import usaddress
from scourgify import normalize_address_record as norm
import zipcodes
import wget ##added
import os

In [121]:
#import package for visual progress bars
%%capture
from tqdm.notebook import tqdm as tqdm
tqdm().pandas()
from IPython.display import clear_output

In [4]:
APIkey = "Insert Your API key for the google geocoder"
#https://developers.google.com/maps/documentation/geocoding/start

In [5]:
gmaps = googlemaps.Client(key=APIkey)

In [6]:
##############################################################################
#https://data.acgov.org/datasets/86b6da3837a34f10b8493ea0d22f517a_0/data
##############################################################################

In [7]:
class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str

In [8]:
# retrieve address points csv from ac gov website
# https://data.acgov.org/datasets/86b6da3837a34f10b8493ea0d22f517a
url = "https://opendata.arcgis.com/datasets/86b6da3837a34f10b8493ea0d22f517a_0.csv"
path = "../data/acgov/ac_gov_address_points.csv"

#delete backup if exists
try:
    os.remove("../data/acgov/ac_gov_address_points_(old).csv")
    print("removing old backup")
except:
    print("no existing backup to clear")
    
#backup most recent version
try:
    os.rename(path, "../data/acgov/ac_gov_address_points_(old).csv")
    print("backing up")
except:
    print("no existing files")

#download most recent version
print("downloading latest version")
x=wget.download(url, path)
print("\n")
print("downloaded: "+x)

removing old backup
no existing files
downloading latest version
-1 / unknown

downloaded: ../data/acgov/ac_gov_address_points.csv


In [9]:
address_points = pandas.read_csv("../data/acgov/ac_gov_address_points.csv", converters=StringConverter())
address_points = address_points[["ST_NUM","FEANME", "FEATYP", "CITY", "ZIPCODE", "AddressLookup", "APN"]]
address_points = address_points.rename(columns=
{
    "ST_NUM":"num",
    "FEANME":"street",
    "FEATYP":"type",
    "CITY":"city",
    "ZIPCODE":"zip",
    "AddressLookup":"address"
})
address_points.head()

,num,street,type,city,zip,address,APN
0,1451,159TH,AV,SAN LEANDRO,94578,1451 159TH AV SAN LEANDRO 94578,080 004007200
1,1459,159TH,AV,SAN LEANDRO,94578,1459 159TH AV SAN LEANDRO 94578,080 004007200
2,1445,159TH,AV,SAN LEANDRO,94578,1445 159TH AV SAN LEANDRO 94578,080 004007304
3,15850,14TH,ST,SAN LEANDRO,94578,15850 E 14TH ST SAN LEANDRO 94578,080 004007810
4,1447,159TH,AV,SAN LEANDRO,94578,1447 159TH AV SAN LEANDRO 94578,080 004007304


In [10]:
#only want east oakland area
targetZips = ["94601", "94603", "94621"]
east_oakland_addr = address_points[[i in targetZips for i in address_points['zip']]]
len(east_oakland_addr)

27182

In [11]:
east_oakland_addr.reset_index(inplace=True)
east_oakland_addr.drop('index', axis=1, inplace=True)
east_oakland_addr.head()

C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,num,street,type,city,zip,address,APN
0,1025,47TH,AV,OAKLAND,94601,1025 47TH AV OAKLAND 94601,034 226100104
1,4635,12TH,ST,OAKLAND,94601,4635 E 12TH ST OAKLAND 94601,034 226100103
2,1046,45TH,AV,OAKLAND,94601,1046 45TH AV OAKLAND 94601,034 226201102
3,1038,45TH,AV,OAKLAND,94601,1038 45TH AV OAKLAND 94601,034 226201102
4,1054,45TH,AV,OAKLAND,94601,1054 45TH AV OAKLAND 94601,034 226201200


In [14]:
# retrieve parcels csv from ac gov website
# https://data.acgov.org/datasets/b55c25ae04fc47fc9c188dbbfcd51192_0/
url = "https://opendata.arcgis.com/datasets/b55c25ae04fc47fc9c188dbbfcd51192_0.csv"
path = "../data/acgov/ac_gov_parcels.csv"

#delete backup if exists
try:
    os.remove("../data/acgov/ac_gov_parcels_(old).csv")
    print("removing old backup")
except:
    print("no existing backup to clear")
    
#backup most recent version
try:
    os.rename(path, "../data/acgov/ac_gov_parcels_(old).csv")
    print("backing up")
except:
    print("no existing files")

#download most recent version
print("downloading latest version")
x=wget.download(url, path)
print("\n")
print("downloaded: "+x)

removing old backup
backing up
downloading latest version
-1 / unknown

downloaded: ../data/acgov/ac_gov_parcels.csv


In [17]:
parcels = pandas.read_csv("../data/acgov/ac_gov_parcels.csv", converters=StringConverter())
parcels = parcels[['APN','APN_SORT', 'YearBuilt', 'SitusStreetNumber', 'SitusStreetName', 'SitusUnit', 'SitusCity', 'SitusZip', 'SitusAddress','MailingAddress']]
print(len(parcels))
parcels.tail()

482339


,APN,APN_SORT,YearBuilt,SitusStreetNumber,SitusStreetName,SitusUnit,SitusCity,SitusZip,SitusAddress,MailingAddress
482334,99B-4901-6-5,099B490100605,0,,VASCO RD,,LIVERMORE,94550,VASCO RD LIVERMORE 94550,PO BOX 29246 PHOENIX AZ 85038
482335,99B-4901-1-7,099B490100107,0,4763,VASCO RD,,LIVERMORE,94550,4763 VASCO RD LIVERMORE 94550,2443 FAIR OAKS BLVD 311 SACRAMENTO CA 95825
482336,99B-4901-2-4,099B490100204,1980,4135,N VASCO RD,,LIVERMORE,94551,4135 N VASCO RD LIVERMORE 94551,2443 FAIR OAKS BLVD 311 SACRAMENTO CA 95825
482337,99B-4901-5,099B490100500,1915,3661,N VASCO RD,,LIVERMORE,94551,3661 N VASCO RD LIVERMORE 94551,3661 N VASCO RD LIVERMORE CA 94551
482338,99B-4901-7-2,099B490100702,0,,VASCO RD,,LIVERMORE,94550,VASCO RD LIVERMORE 94550,4458 TARCENTO LN DUBLIN CA 94568


In [18]:
#only want east oakland area
targetZips = ["94601", "94603", "94621"]
east_oakland_parcels = parcels[[i in targetZips for i in parcels['SitusZip']]]
len(east_oakland_parcels)

26250

In [19]:
east_oakland_parcels.reset_index(inplace=True)
east_oakland_parcels.drop('index', axis=1, inplace=True)
east_oakland_parcels.head()

,APN,APN_SORT,YearBuilt,SitusStreetNumber,SitusStreetName,SitusUnit,SitusCity,SitusZip,SitusAddress,MailingAddress
0,14-1199-34,014 119903400,1983,55TH,ST,,OAKLAND,94621,55TH ST OAKLAND 94621,835 55TH ST OAKLAND CA 94608
1,19-98-5-6,019 009800506,0,2434,E 11TH ST,,OAKLAND,94601,2434 E 11TH ST OAKLAND 94601,PO BOX 7509 OAKLAND CA 94601
2,19-98-6,019 009800600,0,1134,MILLER AVE,,OAKLAND,94601,1134 MILLER AVE OAKLAND 94601,PO BOX 7509 OAKLAND CA 94601
3,19-98-5-5,019 009800505,1900,2424,E 11TH ST,,OAKLAND,94601,2424 E 11TH ST OAKLAND 94601,P.O. BOX 3999 BERKELEY CA 94703
4,19-98-5-4,019 009800504,1900,2430,E 11TH ST,,OAKLAND,94601,2430 E 11TH ST OAKLAND 94601,2430 E 11TH ST OAKLAND CA 94601


In [ ]:
a#############################################################################

In [20]:
#takes zipcode as string input, returns name of city
def zipToCity(zipCode):
    return zipcodes.matching(zipCode)[0]["city"]

#takes address (as much as exists...hopefully enough) as string input, returns zipcode
def retrieveZip(address):
    res = gmaps.geocode(address)
    zipcode =  res[0]['address_components'][-1]['long_name']
    return zipcode

In [235]:
#usaddress.tag breaks up the address into components
    #some addresses have unusual elements (ex/ 'StreetNamePreDirectional' as in 10500 E 14th St)
    #if an address doesn't work, you might need to update this function which is brittle
    #a more robust option is the google geocoder to parse and normalize addresses, but $$$
#scourgify.norm converts the input into a standard format
def parse_and_norm(address, recursive=False):
    if recursive:
        print("Attempting to parse {} with google geocoder".format(address))
    try:
        parsed = usaddress.tag(address)[0]

        #AddressNumberPrefix
        AddressNumber = parsed['AddressNumber']
        #AddressNumberSuffix
        StreetNamePreDirectional = " " + parsed['StreetNamePreDirectional'] if 'StreetNamePreDirectional' in parsed else ""
        StreetName = parsed["StreetName"] if 'StreetName' in parsed else ""
        StreetNamePostType = parsed['StreetNamePostType'] if 'StreetNamePostType' in parsed else ""
        OccupancyType = parsed['OccupancyType'] if 'OccupancyType' in parsed else "" #ex/Apt, Suite
        OccupancyIdentifier = parsed['OccupancyIdentifier'] if 'OccupancyIdentifier' in parsed else "" #ex/ #3
        PlaceName = parsed['PlaceName'] if 'Placename' in parsed else ""
        ZipCode = parsed['ZipCode'] if 'ZipCode' in parsed else ""

        #failure if missing AddressNumber, StreetName, or PlaceName&&ZipCode
        if (AddressNumber=="") or (StreetName=="") or (PlaceName=="" and ZipCode==""):
            raise Exception("failed to parse and normalize address")

        #retrieve missing zip with google geocoder (this should be rarely needed)
        if len(ZipCode) < 5:
            print("Invalid Zip Code ({}) from {}".format(ZipCode,address))
            ZipCode = retrieveZip(address)
            if ZipCode == -1: #error
                return {}

        #retrieve missing city name with zip
        if len(PlaceName) < 2 and ZipCode!="":
            PlaceName = zipToCity(ZipCode)
            
        #known error with norm; doesn't standardize BLVD to BL
        if StreetNamePostType.lower() == "blvd":
            StreetNamePostType = "BL"

        input = {
            'address_line_1': AddressNumber + StreetNamePreDirectional + " " + StreetName + " " + StreetNamePostType,
            'address_line_2': "",
            'city': PlaceName,
            'state': 'CA', #hardcoded
            'postal_code': ZipCode
        }

        normalized = norm(input)
        if (recursive):
            print("     ^ success")
        return normalized
    except:
        #if failed, try to use google geocoder api to clean address and recover missing fields
        if recursive==True:
            print("failed to parse and normalize address -- google geocoder failed too")
            return {}
        try:
            google_address = gmaps.geocode(address)[0]['formatted_address']
            return parse_and_norm(google_address, True)
        except:
            print("failed to parse and normalize address -- google geocoder failed too")
            return {}

In [119]:
#Parse and Normalize address column to prepare for lookup from scraped data
#input table and name of address column
#return list (index, address) of rows which could not be standardized
def standardize_address(tbl, col):
    errors = []
    for i in tqdm(list(tbl.index)):
        try:
            if (i%1000==0): print(i)
            normed = parse_and_norm(tbl.loc[i,col])
            tbl.loc[i,"line_1"]=normed['address_line_1']
            tbl.loc[i,'line_2']=normed['address_line_2']
            tbl.loc[i,'city']=normed['city']
            tbl.loc[i,'state']=normed['state']
            tbl.loc[i,'postal']=normed['postal_code']
        except Exception as e:
            #print(i)
            #print(repr(e))
            errors.append(i)
    print("Errors: " + str(len(errors)))
    failingAddresses = [(i,tbl.loc[i,col]) for i in errors]
    return failingAddresses

In [241]:
e=standardize_address(east_oakland_addr, "address")

HBox(children=(FloatProgress(value=0.0, max=27182.0), HTML(value='')))

0


C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000

Errors: 0


In [242]:
#This dataset has many entry errors with missing street numbers; either blank or street name inputted instead of number
bad_data = east_oakland_parcels[[not i.isnumeric() for i in east_oakland_parcels['SitusStreetNumber']]]
bad_data.head()

,APN,APN_SORT,YearBuilt,SitusStreetNumber,SitusStreetName,SitusUnit,SitusCity,SitusZip,SitusAddress,MailingAddress,line_1,line_2,city,state,postal


In [243]:
len(east_oakland_parcels)

24442

In [244]:
#remove bad entries from east_oakland_parcels
east_oakland_parcels = east_oakland_parcels[[ i.isnumeric() for i in east_oakland_parcels['SitusStreetNumber']]]
len(east_oakland_parcels)

24442

In [245]:
e = standardize_address(east_oakland_parcels, "SitusAddress")

HBox(children=(FloatProgress(value=0.0, max=24442.0), HTML(value='')))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000

Errors: 0


In [246]:
east_oakland_addr.to_csv("../data/processed_acgov/eo_addr.csv", index=False)
east_oakland_parcels.to_csv("../data/processed_acgov/eo_parcels.csv", index=False)

In [247]:
###################################################################################
# Use above data and functions to retrieve APNs from address strings
###################################################################################

In [248]:
eo_addr = pandas.read_csv("../data/processed_acgov/eo_addr.csv", converters=StringConverter())
eo_parcels = pandas.read_csv("../data/processed_acgov/eo_parcels.csv", converters=StringConverter())

In [252]:
a="1467 Fruitvale AVE, APT 141, OAKLAND CA"

In [253]:
parse_and_norm(a)

Attempting to parse 1467 Fruitvale Ave #141, Oakland, CA 94601, USA with google geocoder
     ^ success


{'address_line_1': '1467 FRUITVALE AVE',
 'address_line_2': None,
 'city': 'OAKLAND',
 'state': 'CA',
 'postal_code': '94601'}

In [254]:
getAPN(a)

Attempting to parse 1467 Fruitvale Ave #141, Oakland, CA 94601, USA with google geocoder
     ^ success


[]

In [255]:
east_oakland_addr[east_oakland_addr['num']=="1467"]

,num,street,type,city,zip,address,APN,line_1,line_2,state,postal
3008,1467,82ND,AV,OAKLAND,94621,1467 82ND AV B OAKLAND 94621,040 336400900,1467 82ND AVE,NaN,CA,94621
3009,1467,82ND,AV,OAKLAND,94621,1467 82ND AV OAKLAND 94621,040 336400900,1467 82ND AVE,NaN,CA,94621
5034,1467,70TH,AV,OAKLAND,94621,1467 70TH AV OAKLAND 94621,039 325000400,1467 70TH AVE,NaN,CA,94621
7098,1467,83RD,AV,OAKLAND,94621,1467 83RD AV OAKLAND 94621,043 455000200,1467 83RD AVE,NaN,CA,94621
7185,1467,73RD,AV,OAKLAND,94621,1467 73RD AV OAKLAND 94621,039 331101400,1467 73RD AVE,NaN,CA,94621
8594,1467,74TH,AV,OAKLAND,94621,1467 74TH AV OAKLAND 94621,040 331702200,1467 74TH AVE,NaN,CA,94621
11287,1467,84TH,AV,OAKLAND,94621,1467 84TH AV OAKLAND 94621,043 455100200,1467 84TH AVE,NaN,CA,94621
25997,1467,79TH,AV,OAKLAND,94621,1467 79TH AV OAKLAND 94621,040 336100800,1467 79TH AVE,NaN,CA,94621


In [256]:
east_oakland_parcels[east_oakland_parcels['SitusStreetNumber']=="1467"]

,APN,APN_SORT,YearBuilt,SitusStreetNumber,SitusStreetName,SitusUnit,SitusCity,SitusZip,SitusAddress,MailingAddress,line_1,line_2,city,state,postal
10582,39-3311-14,039 331101400,1924,1467,73RD AVE,,OAKLAND,94621,1467 73RD AVE OAKLAND 94621,1467 73RD AVE OAKLAND CA 94621,1467 73RD AVE,NaN,OAKLAND,CA,94621
11121,39-3250-4,039 325000400,1929,1467,70TH AVE,,OAKLAND,94621,1467 70TH AVE OAKLAND 94621,1467 70TH AVE OAKLAND CA 94621,1467 70TH AVE,NaN,OAKLAND,CA,94621
11717,40-3364-9,040 336400900,1959,1467,82ND AVE,,OAKLAND,94621,1467 82ND AVE OAKLAND 94621,4011 ARDLEY AVE 1 OAKLAND CA 94602,1467 82ND AVE,NaN,OAKLAND,CA,94621
11761,40-3361-8,040 336100800,1919,1467,79TH AVE,,OAKLAND,94621,1467 79TH AVE OAKLAND 94621,2139 LONGLEAF CIR SAN RAMON CA 94582,1467 79TH AVE,NaN,OAKLAND,CA,94621
12356,40-3317-22,040 331702200,1924,1467,74TH AVE,,OAKLAND,94621,1467 74TH AVE OAKLAND 94621,4849 E 12TH ST OAKLAND CA 94601,1467 74TH AVE,NaN,OAKLAND,CA,94621
16636,43-4550-2,043 455000200,1943,1467,83RD AVE,,OAKLAND,94621,1467 83RD AVE OAKLAND 94621,1467 83RD AVE OAKLAND CA 94621,1467 83RD AVE,NaN,OAKLAND,CA,94621
23257,47-5596-68,047 559607600,2002,1467,TUCKER ST,,OAKLAND,94603,1467 TUCKER ST OAKLAND 94603,1467 TUCKER ST OAKLAND CA 94603,1467 TUCKER ST,NaN,OAKLAND,CA,94603


In [226]:
normalized = parse_and_norm("10950 INTERNATIONAL BL, Oakland CA 94603")

In [228]:
line_1 = normalized['address_line_1']
postal = normalized['postal_code']
q1 = eo_addr.query("line_1==@line_1 and postal==@postal").drop_duplicates("APN")["APN"]
q2 = eo_parcels.query("line_1==@line_1 and postal==@postal").drop_duplicates("APN_SORT")["APN_SORT"]

In [229]:
line_1

'10950 INTERNATIONAL BL'

In [165]:
#Retrieve APN From Address string
#returns series; address may have multiple parcels
def getAPN(address):
    normalized = parse_and_norm(address)
    line_1 = normalized['address_line_1']
    postal = normalized['postal_code']
    q1 = eo_addr.query("line_1==@line_1 and postal==@postal").drop_duplicates("APN")["APN"]
    q2 = eo_parcels.query("line_1==@line_1 and postal==@postal").drop_duplicates("APN_SORT")["APN_SORT"]
    #convert list to set and back to only return unique values (set keys must be unique)
    return list(set(q1.tolist() + q2.tolist()))
        

In [ ]:
getAPN("1745 CHURCH ST, Oakland CA 94621")